In [ ]:
import idom

In [ ]:
from example_utils import localhost

webpage_url = localhost('http', 8765) + '/idom/client/index.html'
websocket_url = localhost('ws', 8765) + '/idom/stream'

print('Get a webpage that streams layout updates via a websocket:')
print(webpage_url)
print()
print('Websockets use this route to stream layout updates:')
print(websocket_url)

In [ ]:
import idom

@idom.element
async def Slideshow(self, index=0):
    events = idom.Events()

    @events.on("click")
    def change():
        self.update(index + 1)

    url = f"https://picsum.photos/800/300?image={index}"
    return idom.node("img", src=url, eventHandlers=events)

idom.SimpleServer(Slideshow).daemon("0.0.0.0", 8765)

In [ ]:
idom.display("jupyter", websocket_url)